In [1]:
from dotenv import load_dotenv
from enum import Enum
import instructor
from openai import OpenAI
import os
from pathlib import Path
from pprint import pprint as pp
from pydantic import BaseModel, Field, StringConstraints, conlist, field_validator
from typing import List, Optional, Union, Dict
from typing_extensions import Annotated

In [ ]:
# OWL


class OntologyClass(BaseModel):
    name: str
    label: Optional[str] = None
    subclass_of: Optional[List[str]] = None
    equivalent_to: Optional[List[str]] = None
    disjoint_with: Optional[List[str]] = None

class ObjectProperty(BaseModel):
    name: str
    label: Optional[str] = None
    domain: Optional[str] = None
    range: Optional[str] = None
    inverse_of: Optional[str] = None

class DataProperty(BaseModel):
    name: str
    label: Optional[str] = None
    domain: Optional[str] = None
    range: Optional[str] = None

class AnnotationProperty(BaseModel):
    name: str
    label: Optional[str] = None

class Individual(BaseModel):
    class_name: str
    instance_id: str
    properties: Optional[dict] = None

class Axiom(BaseModel):
    type: str  # 'class', 'property', 'individual'
    content: dict

class Ontology(BaseModel):
    classes: List[OntologyClass] = []
    object_properties: List[ObjectProperty] = []
    data_properties: List[DataProperty] = []
    annotation_properties: List[AnnotationProperty] = []
    individuals: List[Individual] = []
    axioms: List[Axiom] = []

In [2]:
# revised OWL


class PropertyBase(BaseModel):
    """Base model for common property fields."""
    name: Annotated[str, Field(..., description="The unique identifier for the property. This should be a non-empty string that clearly identifies the property within the ontology.")]
    label: Annotated[Optional[str], Field(None, description="A human-readable label for the property. This should provide a clear, descriptive name that conveys the purpose of the property.")]
    domain: Annotated[Optional[str], Field(None, description="The domain class of the property. This should specify the class that the property applies to, ensuring clarity in the ontology.")]
    range: Annotated[Optional[str], Field(None, description="The range class or datatype of the property. This should specify the type of values the property can have, providing clear constraints.")]

class OntologyClass(BaseModel):
    """Model representing an ontology class."""
    name: Annotated[str, Field(..., description="The unique identifier for the ontology class. This should be a non-empty string that clearly identifies the class within the ontology.")]
    label: Annotated[Optional[str], Field(None, description="A human-readable label for the class. This should provide a clear, descriptive name that conveys the purpose of the class.")]
    subclass_of: Annotated[Optional[List[str]], Field(None, description="A list of classes this class is a subclass of. This should specify parent classes, ensuring proper hierarchical structure.",
                                                      example=["ParentClass1", "ParentClass2"])]
    equivalent_to: Annotated[Optional[List[str]], Field(None, description="A list of classes this class is semantically equivalent to. This should provide equivalent class names, ensuring clarity in class relationships.",
                                                        example=["EquivalentClass1", "EquivalentClass2"])]
    disjoint_with: Annotated[Optional[List[str]], Field(None, description="A list of classes that are disjoint with this class. This should specify classes that cannot overlap with this class, ensuring proper class distinctions.",
                                                        example=["DisjointClass1", "DisjointClass2"])]

class ObjectProperty(PropertyBase):
    """Model representing an object property."""
    inverse_of: Annotated[Optional[str], Field(None, description="The inverse property of this object property, if any. This should specify the property that represents the inverse relationship, ensuring clarity in bidirectional relationships.",
                                               example="inverseProperty")]

class DataProperty(PropertyBase):
    """Model representing a data property."""
    range: Annotated[Optional[str], Field(None, description="The range datatype of the data property. This should specify the type of values the property can have, providing clear constraints.",
                                          example="xsd:string")]

class AnnotationProperty(BaseModel):
    """Model representing an annotation property."""
    name: Annotated[str, Field(..., description="The unique identifier for the annotation property. This should be a non-empty string that clearly identifies the property within the ontology.")]
    label: Annotated[Optional[str], Field(None, description="A human-readable label for the property. This should provide a clear, descriptive name that conveys the purpose of the property.",
                                          example="Annotation Label")]

class Individual(BaseModel):
    """Model representing an individual instance."""
    class_name: Annotated[str, Field(..., description="The class that the individual instance belongs to. This should specify the class name, ensuring clarity in individual classification.",
                                     example="Person")]
    instance_id: Annotated[str, Field(..., description="The unique identifier for the individual instance. This should be a non-empty string that clearly identifies the individual within the ontology.",
                                      example="person_123")]
    properties: Annotated[Optional[Dict[str, Union[str, int, float, bool]]], Field(None, description="A dictionary of property names and their corresponding values for the individual. This should provide detailed property values, ensuring a comprehensive representation of the individual.",
                                                                                   example={"age": 30, "name": "John Doe"})]

class AxiomType(str, Enum):
    """Enum for axiom types."""
    CLASS = "class"
    PROPERTY = "property"
    INDIVIDUAL = "individual"

class Axiom(BaseModel):
    """Model representing an axiom."""
    type: Annotated[AxiomType, Field(..., description="The type of axiom, such as 'class', 'property', or 'individual'. This should clearly specify the axiom type, ensuring proper categorization.",
                                     example="class")]
    content: Annotated[Dict, Field(..., description="The content of the axiom represented as a dictionary. This should provide detailed content, ensuring a comprehensive representation of the axiom.",
                                   example={"class": "Person", "property": "age", "value": 30})]

class Ontology(BaseModel):
    """Model representing an ontology."""
    classes: Annotated[List[OntologyClass], Field(..., description="A list of all ontology classes. This should provide detailed class definitions, ensuring a comprehensive representation of the ontology.")]
    object_properties: Annotated[List[ObjectProperty], Field(..., description="A list of all object properties. This should provide detailed property definitions, ensuring a comprehensive representation of the ontology.")]
    data_properties: Annotated[List[DataProperty], Field(..., description="A list of all data properties. This should provide detailed property definitions, ensuring a comprehensive representation of the ontology.")]
    annotation_properties: Annotated[List[AnnotationProperty], Field(..., description="A list of all annotation properties. This should provide detailed property definitions, ensuring a comprehensive representation of the ontology.")]
    individuals: Annotated[List[Individual], Field(..., description="A list of all individual instances. This should provide detailed individual definitions, ensuring a comprehensive representation of the ontology.")]
    axioms: Annotated[List[Axiom], Field(..., description="A list of all axioms. This should provide detailed axiom definitions, ensuring a comprehensive representation of the ontology.")]




In [3]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL = "gpt-4o" # points to latest GPT model

#instantiate client
client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [4]:
# wrapper

def wrapper(system_prompt: str | None = None, user_context: str | list = None, response_model: BaseModel | None = None, max_retries: int = 3):
    """Wrapper function to generate LLM completion"""
    messages = []
    if system_prompt is not None:
        messages.append({"role": "system", "content": system_prompt})
    if user_context is not None:
        messages.append({"role": "user", "content": user_context})

    completion = client.chat.completions.create(
        model=GPT_MODEL,
        response_model=response_model,
        max_retries=max_retries,
        messages=messages
    )
    return completion

In [17]:
system_prompt = """
You are a world class ontology algorithm capable of systematically extracting or inferring an ontology from a given context.
You are an expert at extracting or inferring detailed lists of ontology classes, properties, individuals and axioms. 
Each item and each list is precise, granular, comprehensive and complete.
Your purpose is to teach this ontology to a skilled individual without the domain knowledge.
Generate a dense, rich articulation with encyclopedic breadth and depth of an OWL ontology.
Always think broadly about a wide variety of aspects and factors then think deeply about each factor and aspect.
Before you call the function, think step-by-step to get a better understanding of the problem.
"""

In [21]:
user_context = "Tell me everything about hinduism"

In [22]:
response = wrapper(system_prompt=system_prompt, user_context=user_context, response_model=Ontology)

In [23]:
# print response
response.dict()


{'classes': [{'name': 'Deity',
   'label': 'Deity',
   'subclass_of': ['SupernaturalBeing'],
   'equivalent_to': None,
   'disjoint_with': ['Human']},
  {'name': 'Scripture',
   'label': 'Scripture',
   'subclass_of': ['Text'],
   'equivalent_to': None,
   'disjoint_with': ['OralTradition']},
  {'name': 'Ritual',
   'label': 'Ritual',
   'subclass_of': None,
   'equivalent_to': None,
   'disjoint_with': None},
  {'name': 'Festival',
   'label': 'Festival',
   'subclass_of': None,
   'equivalent_to': None,
   'disjoint_with': None},
  {'name': 'Philosophy',
   'label': 'Philosophy',
   'subclass_of': None,
   'equivalent_to': None,
   'disjoint_with': None},
  {'name': 'SchoolOfThought',
   'label': 'School of Thought',
   'subclass_of': ['Philosophy'],
   'equivalent_to': None,
   'disjoint_with': None},
  {'name': 'Temple',
   'label': 'Temple',
   'subclass_of': None,
   'equivalent_to': None,
   'disjoint_with': None},
  {'name': 'Human',
   'label': 'Human',
   'subclass_of': None,